# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [83]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

from pprint import pprint

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [84]:
cities_data_file = "../WeatherPy/output_data/cities.csv"
cities_df = pd.read_csv(cities_data_file)
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,bay roberts,47.60,-53.26,37.40,100,90,8.05,CA,1603862103
1,laguna,38.42,-121.42,57.00,58,1,3.36,US,1603861947
2,hambantota,6.12,81.12,84.87,67,94,6.73,LK,1603861997
3,narsaq,60.92,-46.05,30.20,80,75,1.12,GL,1603862104
4,castro,-24.79,-50.01,55.54,76,0,3.20,BR,1603862104


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [85]:
# Get the locations and Humidity
locations = cities_df[["Lat","Lng"]].astype('float')
humidity = cities_df["Humidity"].astype('float')

gmaps.configure(api_key=g_key)

fig = gmaps.figure(center=(45.5051064,-122.6750261), zoom_level=2)

heat_layer = gmaps.heatmap_layer(
    locations,
    weights=humidity,
    dissipating=False,
    max_intensity=100,
    point_radius = 1.25    
)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [86]:
ideal_vacation_weather = cities_df[
    (cities_df["Max Temp"] < 80) & 
    (cities_df["Max Temp"] > 65) &
    (cities_df["Wind Speed"] < 10) &
    (cities_df["Humidity"] < 75) &
    (cities_df["Cloudiness"] < 20)
]
ideal_vacation_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
33,arlit,18.74,7.39,73.62,18,3,8.55,NE,1603862113
45,salalah,17.02,54.09,78.80,54,0,2.24,OM,1603862116
71,bandarbeyla,9.49,50.81,78.67,63,4,5.30,SO,1603862125
76,kidal,18.44,1.41,74.50,18,7,7.72,ML,1603862126
111,tessalit,20.20,1.01,73.58,20,0,6.55,ML,1603862136
122,bambous virieux,-20.34,57.76,79.00,65,14,7.00,MU,1603862139
132,kutum,14.20,24.67,70.32,29,0,7.87,SD,1603862142
140,souillac,-20.52,57.52,79.00,65,4,7.00,MU,1603862144
151,upington,-28.45,21.26,73.40,25,0,3.09,ZA,1603862148
164,iranshahr,27.20,60.68,68.00,13,0,1.68,IR,1603862152


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [87]:
hotel_df = ideal_vacation_weather.copy(deep=True).reset_index(drop=True, inplace=False)
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,arlit,18.74,7.39,73.62,18,3,8.55,NE,1603862113,
1,salalah,17.02,54.09,78.80,54,0,2.24,OM,1603862116,
2,bandarbeyla,9.49,50.81,78.67,63,4,5.30,SO,1603862125,
3,kidal,18.44,1.41,74.50,18,7,7.72,ML,1603862126,
4,tessalit,20.20,1.01,73.58,20,0,6.55,ML,1603862136,
5,bambous virieux,-20.34,57.76,79.00,65,14,7.00,MU,1603862139,
6,kutum,14.20,24.67,70.32,29,0,7.87,SD,1603862142,
7,souillac,-20.52,57.52,79.00,65,4,7.00,MU,1603862144,
8,upington,-28.45,21.26,73.40,25,0,3.09,ZA,1603862148,
9,iranshahr,27.20,60.68,68.00,13,0,1.68,IR,1603862152,


In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "keyword": "hotel",
    "types": "lodging",
    "key": g_key,
}

for index, row in hotel_df.iterrows():
    
    lat = row['Lat']
    lng = row["Lng"]
    
    params['location'] = f"{lat},{lng}"
    
    response = requests.get(base_url, params=params)
    
    hotels = response.json()
    
    #pprint(hotels)
    
    try:
        hotel_name = hotels['results'][0]['name']
    except (KeyError, IndexError):
        hotel_name = "No Hotel Found"
        
    hotel_df.loc[index, "Hotel Name"] = hotel_name
    
hotel_df        

In [80]:

fig = gmaps.figure(center=(45.5051064,-122.6750261), zoom_level=2)

heat_layer = gmaps.heatmap_layer(
    locations,
    weights=humidity,
    dissipating=False,
    max_intensity=100,
    point_radius = 1.25    
)

fig.add_layer(heat_layer)

In [81]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [82]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(
    hotel_locations,
    info_box_content=hotel_info
)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))